# Filtrado de DataFrames

<a href="https://colab.research.google.com/github/milocortes/diplomado_ciencia_datos_mide/blob/edicion-2023/notebooks/filtering_dataframe_mide_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a> 

## Optimización del  uso de la memoria  

Al importar un conjunto de datos, es importante considerar si cada columna almacena sus datos en el tipo más óptimo. 

El "mejor" tipo de datos es el que consume menos memoria o proporciona la mayor utilidad.

Por ejemplo, si el conjunto de datos incluye fechas, es ideal importarlas como fechas y horas en lugar de strings, lo que permite operaciones específicas de fecha y hora.

In [1]:
import pandas as pd

pd.read_csv("../datos/employees.csv")

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,8/6/93,NaN,True,Marketing
1,Thomas,Male,3/31/96,61933.0,True,NaN
2,Maria,Female,NaN,130590.0,False,Finance
3,Jerry,NaN,3/4/05,138705.0,True,Finance
4,Larry,Male,1/24/98,101004.0,True,IT
...,...,...,...,...,...,...
996,Phillip,Male,1/31/84,42392.0,False,Finance
997,Russell,Male,5/20/13,96914.0,False,Product
998,Larry,Male,4/20/13,60500.0,False,Business Dev
999,Albert,Male,5/15/12,129949.0,True,Sales



¿Cómo podemos aumentar la utilidad de nuestro conjunto de datos?

Podemos convertir los valores de texto en la columna <code>Start Date</code> a <code>datetimes</code> con el parámetro <code>parse_dates</code>:

In [2]:
employees = pd.read_csv("../datos/employees.csv", parse_dates = ["Start Date"]).head()
employees

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,NaN,True,Marketing
1,Thomas,Male,1996-03-31,61933.0,True,NaN
2,Maria,Female,NaT,130590.0,False,Finance
3,Jerry,NaN,2005-03-04,138705.0,True,Finance
4,Larry,Male,1998-01-24,101004.0,True,IT



Hay algunas opciones disponibles para mejorar la velocidad y la eficiencia de las operaciones de <code>DataFrame</code>.

Podemos invocar el método <code>info</code> para ver una lista de las columnas, sus tipos de datos, un recuento de los valores faltantes y el consumo total de memoria del <code>DataFrame</code>:

In [3]:
employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   First Name  5 non-null      object        
 1   Gender      4 non-null      object        
 2   Start Date  4 non-null      datetime64[ns]
 3   Salary      4 non-null      float64       
 4   Mgmt        5 non-null      object        
 5   Team        4 non-null      object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 368.0+ bytes


## Convertir tipos de datos con el método <code>astype</code>

Pandas importó los valores de la columna Mgmt como cadenas. 

La columna almacena solo dos valores: True y False. 

Podemos reducir el uso de la memoria convirtiendo los valores al tipo de datos booleano más ligero.

El método <code>astype</code> convierte los valores de una <code>Series</code> a un tipo de datos diferente.

In [4]:
employees["Mgmt"].astype(bool)

0     True
1     True
2    False
3     True
4     True
Name: Mgmt, dtype: bool

In [5]:
employees["Mgmt"] = employees["Mgmt"].astype(bool)

In [6]:
employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   First Name  5 non-null      object        
 1   Gender      4 non-null      object        
 2   Start Date  4 non-null      datetime64[ns]
 3   Salary      4 non-null      float64       
 4   Mgmt        5 non-null      bool          
 5   Team        4 non-null      object        
dtypes: bool(1), datetime64[ns](1), float64(1), object(3)
memory usage: 333.0+ bytes


Sin embargo, en <code>employees</code>, pandas almacena los valores de <code>Salary</code> en flotantes. Para admitir los <code>NaN</code> en toda la columna, pandas convierte los números enteros en números de punto flotante, un requisito técnico de la biblioteca que vimos anteriormente.

In [7]:
employees["Salary"].fillna(0).astype(int)

0         0
1     61933
2    130590
3    138705
4    101004
Name: Salary, dtype: int64

In [8]:
employees["Salary"] = employees["Salary"].fillna(0).astype(int)

In [9]:
employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   First Name  5 non-null      object        
 1   Gender      4 non-null      object        
 2   Start Date  4 non-null      datetime64[ns]
 3   Salary      5 non-null      int64         
 4   Mgmt        5 non-null      bool          
 5   Team        4 non-null      object        
dtypes: bool(1), datetime64[ns](1), int64(1), object(3)
memory usage: 333.0+ bytes



Podemos hacer una optimización adicional.


Pandas incluye un tipo de datos especial llamado *category*, que es ideal para una columna que consta de una pequeña cantidad de valores únicos en relación con su tamaño total.

Algunos ejemplos cotidianos de datos con un número limitado de valores incluyen género, días de la semana, tipos de sangre, planetas y grupos de ingresos. 

Detrás de escena, pandas almacena solo una copia de cada valor categórico en lugar de almacenar duplicados en filas.

In [10]:
employees.nunique()

First Name    5
Gender        2
Start Date    4
Salary        5
Mgmt          2
Team          3
dtype: int64

In [11]:
employees["Gender"].astype("category")

0      Male
1      Male
2    Female
3       NaN
4      Male
Name: Gender, dtype: category
Categories (2, object): ['Female', 'Male']

In [12]:
employees["Gender"] = employees["Gender"].astype("category")

In [13]:
employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   First Name  5 non-null      object        
 1   Gender      4 non-null      category      
 2   Start Date  4 non-null      datetime64[ns]
 3   Salary      5 non-null      int64         
 4   Mgmt        5 non-null      bool          
 5   Team        4 non-null      object        
dtypes: bool(1), category(1), datetime64[ns](1), int64(1), object(2)
memory usage: 422.0+ bytes


In [14]:
employees["Team"] = employees["Team"].astype("category")
employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   First Name  5 non-null      object        
 1   Gender      4 non-null      category      
 2   Start Date  4 non-null      datetime64[ns]
 3   Salary      5 non-null      int64         
 4   Mgmt        5 non-null      bool          
 5   Team        4 non-null      category      
dtypes: bool(1), category(2), datetime64[ns](1), int64(1), object(1)
memory usage: 519.0+ bytes


## Filtrado por una sola condición
La extracción de un subconjunto de datos es quizás la operación más común en el análisis de datos. Un *subconjunto* es una porción de un conjunto de datos más grande que se ajusta a algún tipo de condición.


Supongamos que queremos generar una lista de todos los empleados llamados "<code>Maria</code>" .

Para realizar esta tarea, debemos filtrar el conjunto de datos de nuestros empleados en función de los valores de la columna <code>First Name</code>. 

Para comparar cada entrada de <code>Series</code> con un valor constante, colocamos <code>Series</code> en un lado del operador de igualdad y el valor en el otro:

In [ ]:
Series == value

Cuando comparamos una <code>Series</code> con un operador de igualdad, pandas devuelve una <code>Series</code> de valores booleanos.

In [16]:
employees["First Name"] == "Maria"

0    False
1    False
2     True
3    False
4    False
Name: First Name, dtype: bool

Pandas ofrece una sintaxis conveniente para extraer filas usando una <code>Series</code> de booleanos.


Para filtrar filas, proporcionamos el booleano <code>Series</code> entre corchetes después del <code>DataFrame</code> :

In [17]:
employees[employees["First Name"] == "Maria"]

,First Name,Gender,Start Date,Salary,Mgmt,Team
2,Maria,Female,NaT,130590,False,Finance



Si el uso de corchetes múltiples es confuso, es posible asignar el booleano <code>Series</code> a una variable  y luego pasar esa variable entre corchetes.

In [18]:
marias = employees["First Name"] == "Maria"
employees[marias]

,First Name,Gender,Start Date,Salary,Mgmt,Team
2,Maria,Female,NaT,130590,False,Finance



Probemos con otro ejemplo. ¿Qué pasa si queremos extraer un subconjunto de empleados que no están en el equipo de Finanzas? El protocolo sigue siendo el mismo, pero con un ligero cambio.

In [19]:
employees["Team"] != "Finance"

0     True
1     True
2    False
3    False
4     True
Name: Team, dtype: bool

In [20]:
employees[employees["Team"] != "Finance"]

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,0,True,Marketing
1,Thomas,Male,1996-03-31,61933,True,NaN
4,Larry,Male,1998-01-24,101004,True,IT


¿Qué pasa si queremos recuperar todos los gerentes de la empresa? Los administradores tienen un valor de <code>True</code> en la columna Mgmt.

Podríamos ejecutar <code>employees["Mgmt"]==True</code> , pero no es necesario porque Mgmt ya es una <code>Series</code> de booleanos.

In [21]:
employees[employees["Mgmt"]]

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,0,True,Marketing
1,Thomas,Male,1996-03-31,61933,True,NaN
3,Jerry,NaN,2005-03-04,138705,True,Finance
4,Larry,Male,1998-01-24,101004,True,IT


El siguiente ejemplo genera una <code>Series</code> booleana para valores de <code>Salary</code> superiores a $100 000:

In [22]:
high_earners = employees["Salary"] > 100000
employees[high_earners].head()

,First Name,Gender,Start Date,Salary,Mgmt,Team
2,Maria,Female,NaT,130590,False,Finance
3,Jerry,NaN,2005-03-04,138705,True,Finance
4,Larry,Male,1998-01-24,101004,True,IT


## Filtrado por múltiples condiciones

Podemos filtrar un <code>DataFrame</code> con múltiples condiciones creando dos <code>Series</code> booleanas independientes y luego declarando el criterio lógico que Pandas deberían aplicar entre ellos.

### La condición AND

Supongamos que queremos encontrar a todas las empleadas que trabajan en el equipo de *business development*.

Ahora Pandas deben buscar dos condiciones para filtar una fila: un valor de "<code>Female</code>" en la columna <code>Gender</code> y un valor de "<code>Business Dev</code>" en la columna <code>Team</code>.

In [23]:
is_female = employees["Gender"] == "Female"
in_biz_dev = employees["Team"] == "Business Dev"

is_female & in_biz_dev

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [24]:
employees[is_female & in_biz_dev]

,First Name,Gender,Start Date,Salary,Mgmt,Team


Podemos incluir cualquier cantidad de <code>Series</code> entre corchetes, siempre y cuando las separemos con un símbolo <code>&</code>.

In [25]:
is_manager = employees["Mgmt"]
employees[is_female & in_biz_dev & is_manager]

,First Name,Gender,Start Date,Salary,Mgmt,Team


##  La condición OR 

También podemos extraer filas si cumplen una de varias condiciones. No todas las condiciones tienen que
ser True, pero al menos una sí tiene que cumplirla.

Supongamos que queremos identificar a todos los empleados con un <code>Salary</code> inferior a $40 000 o una <code>Start Date</code> posterior al 1 de enero de 2015. 

In [26]:
earning_below_40k = employees["Salary"] < 40000
started_after_2015 = employees["Start Date"] > "2015-01-01"

Usamos un símbolo de barra vertical ( | ) entre las <code>Series</code> booleanas para declarar el criterio <code>OR</code>.

El siguiente ejemplo selecciona las filas en las que cualquiera de las <code>Series</code> booleanas contiene un valor <code>True</code>:

In [27]:
employees[earning_below_40k | started_after_2015].tail()

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,0,True,Marketing


### Inversión con ~

El símbolo de tilde (~) invierte los valores en una <code>Series</code> booleana.

Todos los valores de <code>True</code> se convierten en <code>False</code> y todos los valores de <code>False</code> se convierten en <code>True</code> .

In [28]:
my_series = pd.Series([True, False, True])
my_series

0     True
1    False
2     True
dtype: bool

In [29]:
~my_series

0    False
1     True
2    False
dtype: bool

In [30]:
employees[employees["Salary"] < 100000]

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,0,True,Marketing
1,Thomas,Male,1996-03-31,61933,True,NaN


In [31]:
employees[~(employees["Salary"] >= 100000)]

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,0,True,Marketing
1,Thomas,Male,1996-03-31,61933,True,NaN


# Filtrado por condición

Algunas operaciones de filtrado son más complejas que las simples comprobaciones de igualdad o desigualdad. Afortunadamente, pandas incluye muchos métodos auxiliares que generan series booleanas para este tipo de extracciones.

## El método <code>isin</code>

¿Qué pasa si queremos aislar a los empleados que pertenecen al equipo de Ventas, Legal o Marketing? Podríamos proporcionar tres <code>Series</code> booleanas separadas dentro de los corchetes y agregar el | símbolo para declarar los criterios <code>OR</code>:

In [32]:
sales = employees["Team"]== "Sales"
legal = employees["Team"]== "Legal"
mktg = employees["Team"]== "Marketing"
employees[sales | legal | mktg]

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,0,True,Marketing


Aunque esta solución funciona, no es escalable. ¿Qué pasaría si ahora necesitamos aislar a los empleados de 15 equipos?

Una mejor solución es el método <code>isin</code>, que acepta un objeto iterable (lista, tupla, <code>Series</code>, etc.) y devuelve un booleano <code>Series</code > . <code>True</code> indica que pandas encontró el valor de la fila entre los valores iterables, y <code>False</code> indica que no lo encontró. 


El siguiente ejemplo obtiene el resultado:

In [33]:
all_star_teams = ["Sales", "Legal", "Marketing"]
on_all_star_teams = employees["Team"].isin(all_star_teams)
employees[on_all_star_teams]

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,0,True,Marketing


### Los métodos <code>isnull</code> y <code>notnull</code>


Pandas marca los valores de texto faltantes y los valores numéricos faltantes con una designación <code>NaN</code> (not a number), y marca los valores de fecha y hora faltantes con una designación <code>NaT</code> (not a time)

El método <code>isnull</code> devuelve un booleano <code>Series</code> en el que <code>True</code> indica ausencia de un valor en la fila:

In [34]:
employees["Team"].isnull()

0    False
1     True
2    False
3    False
4    False
Name: Team, dtype: bool

El siguiente ejemplo invoca el método <code>isnull</code> en la columna Fecha de inicio:

In [35]:
employees["Start Date"].isnull()

0    False
1    False
2     True
3    False
4    False
Name: Start Date, dtype: bool

El método <code>notnull</code> devuelve la <code>Series</code> inversa, una en la que <code>True</code> indica que el valor de una fila está presente.

In [36]:
employees["Team"].notnull()

0     True
1    False
2     True
3     True
4     True
Name: Team, dtype: bool

Podemos producir el mismo conjunto de resultados invirtiendo la <code>Series</code> devuelta por el método <code>isnull</code>. Como recordatorio, usamos el símbolo de tilde (~) para invertir una <code>Series</code> booleana:

In [37]:
(~employees["Team"].isnull())

0     True
1    False
2     True
3     True
4     True
Name: Team, dtype: bool

Cualquiera de los enfoques funciona, pero <code>notnull</code> es un poco más descriptivo y se recomienda.

## Manejando valores nulos

Aprendimos a usar el método <code>fillna</code> para reemplazar NaN con un valor constante. Pero también podríamos eliminarlos.

El método <code>dropna</code> elimina las filas del <code>DataFrame</code> que contienen cualquier valor <code>NaN</code>.

No importa cuántos valores falten en una fila; el método excluye la fila si está presente un único <code>NaN</code>.

In [38]:
employees.dropna()

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,0,True,Marketing
4,Larry,Male,1998-01-24,101004,True,IT


Podemos pasar el parámetro <code>how</code> y el argumento de "<code>all</code>" para eliminar las filas en las que faltan todos los valores. Solo una fila en el conjunto de datos, la última, cumple esta condición:

In [39]:
employees.dropna(how = "all")

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,0,True,Marketing
1,Thomas,Male,1996-03-31,61933,True,NaN
2,Maria,Female,NaT,130590,False,Finance
3,Jerry,NaN,2005-03-04,138705,True,Finance
4,Larry,Male,1998-01-24,101004,True,IT


El argumento predeterminado del parámetro <code>how</code> es "<code>any</code>" . Un el argumento "<code>any</code>" elimina una fila si alguno de sus valores está ausente.
    
Podemos usar el parámetro <code>subset</code> para apuntar a filas con un valor faltante en una columna específica.


El siguiente ejemplo elimina las filas a las que les falta un valor en la columna <code>Gender</code>:

In [40]:
employees.dropna(subset = ["Gender"])

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,0,True,Marketing
1,Thomas,Male,1996-03-31,61933,True,NaN
2,Maria,Female,NaT,130590,False,Finance
4,Larry,Male,1998-01-24,101004,True,IT


También podemos pasar el parámetro <code>subset</code> una lista de columnas. Pandas eliminará una fila si falta un valor en cualquiera de las columnas especificadas.

In [41]:
employees.dropna(subset = ["Start Date", "Salary"])

,First Name,Gender,Start Date,Salary,Mgmt,Team
0,Douglas,Male,1993-08-06,0,True,Marketing
1,Thomas,Male,1996-03-31,61933,True,NaN
3,Jerry,NaN,2005-03-04,138705,True,Finance
4,Larry,Male,1998-01-24,101004,True,IT
